__COMP9727 PROJECT:A STEAM GAME RECOMMENDER SYSTEM__


# Introduction
This model is a recommender system for Steam games. It is based on the dataset from Steam, which is a digital distribution platform for video games. The dataset contains 316 games and 21 million users. The recommender system is based on collaborative filtering and content based model. The system integrates the recommender module and a user interaction module.

__PIPELINE__:
1. Data preprocessing, Analysis and Visualization
2. Extracting features from the game dataset, build feature vectors for games. The item vector is a hierarchical structure. First, for the description, use text vectorization technology (such as TF-IDF or Word2Vec) to process the description text of the game to generate a dense numerical vector for each game. For Tags, we still use TF-IDF. But since the game tag describes the game more significantly, we manually increase the TF-IDF weight of the tag.

__Inporting Libraries__

In [1]:
import pandas as pd

__Data Preprocessing__

In [4]:
file_path_reviews = "steam_reviews.csv"
file_path_desc = "desc.csv"
file_path_tags = "tags.csv"

df_reviews = pd.read_csv(file_path_reviews)
df_desc = pd.read_csv(file_path_desc)
df_tags = pd.read_csv(file_path_tags)